# SimCLR (A Simple Framework for Contrastive Learning of Visual Representations)  
- paper review

## Abstract  
- 우리는 어떤 architecture나 memory bank가 없는 contrastive self-supervised learning algorithm을 제안하려 한다  
- 1. data augmentation의 구성요소는 effective predictive task를 정의하는 데 중요한 역할을 한다  
- 2. representation 사이 학습 가능한 nonlinear transformation을 소개하며 contrastive loss는 학습된 representation의 퀄리티를 상당히 개선시킨다  
- 3. contrastive learning은 batch size가 클수록, 더 많이 학습을 할수록 supervised learning보다 성능이 좋다

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure1.png?raw=true" width=400>

## Introduction  
- 대부분 main stream approach들은 generative or discriminative 중 하나다  
- Generative approach는 generate하기 위해 학습하거나 input space에서 모델의 pixel을 학습한다  
- 그러나 pixel-level generation은 연산량 측면에서 expensive하고 representation learning에서 필요하지 않을 수 있다  
- Discriminative approach는 supervised와 비슷하게 objectvie function을 이용해서 representation을 학습한다  
- 그러나 labeling이 필요하다는 단점이 있다  
- 또한 많은 approach들은 pretext task를 디자인하는 데 의존하며 representation의 일반성을 만들어내는 데 한계가 있다  
- 그래서 우리는 SimCLR을 소개하며 이전 SOTA보다 더 좋은 성능을 낸다

## Method    
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure2.png?raw=true" width=400>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/algorithm.png?raw=true" width=450>

### The Contrastive Learning Framework  
- SimCLR은 같은 data example에서 다른 augmentaed view 사이의 일치성을 최대화하도록 contrastive loss를 이용해 학습한다   
- 우리는 다음의 augmentation을 사용했다 
    - random reisize and cropping  
    - random color distortions  
    - random gaussian blur  
    
- neural network base encoder $f(\cdot)$은 augmented data example에서 representation vector를 추출한다  
- 우리의 framework는 어떤 제약 없이 다양한 neural network를 사용하도록 했다  
- 우리는 ResNet을 선택했다  
- small neural network projection head $g(\cdot)$은 representation을 space로 매핑을 시키며 contrastive loss가 사용된다  
- 우리는 $z_i=g\left(\textbf{h}_i\right)=W^{(2)}\sigma\left(W^{(1)}\textbf{h}_i\right)$를 얻기 위해 하나의 hidden layer로 구성되었으며 이때 ReLU 활성화 함수를 사용했다  
- contrastive prediction task는 positive pair $\tilde{\textbf{x}_i}$와 $\tilde{\textbf{x}_j}$가 포함된 set $\left\{\tilde{\textbf{x}}_k\right\}$가 주어지면 $\left\{\tilde{\textbf{x}_{k}}\right\}_{k\neq j}$에 있는 $\tilde{\textbf{x}_j}$를 찾는 것을 목표로 한다  
- 우리는 $N$개의 example 중에서 minibatch를 랜덤으로 sampling하고 augmented example의 pair를 가지고 contrastive prediction task를 정의한다  
- 이때 data point는 총 $2N$개이며 negative sample은 $2(N-1)$개이다  
- $sim\left(\textbf{u}, \textbf{v}\right)=\textbf{u}^{T}\textbf{v}/\Vert\textbf{u}\Vert\Vert\textbf{v}\Vert$는 $\textbf{u}$와 $\textbf{v}$의 $l_2$ norm의 dot product라 하자  
- example $(i,j)$의 positive pair에 대한 loss function은 다음과 같이 정의된다  
$$l_{i,j}=-log\frac{\exp\left(sim\left(\textbf{z}_i,\textbf{z}_j\right)/\tau\right)}{\sum_{k=1}^{2N}\mathcal{1}_{[k\neq i]}\exp\left(sim\left(\textbf{z}_i,\textbf{z}_k\right)/\tau\right)}$$  
- 이때 $\mathcal{1}_{[k\neq i]}\in \left\{0,1\right\}$은 $k\neq i$이면 1로 판단하는 indicator function이고 $\tau$는 temperature parameter이다  

### Training with Large Batch Size  
- framework를 간단하게 하기 위해 우리는 memory bank를 사용하지 않았다  
- 대신, training batch size를 256에서 8192로 다양하게 적용했다  
- optimizer로 SGD나 Momentum을 사용할 경우 large batch size는 불안정하게 학습한다  
- 이를 안정화 시키기 위해 우리는 LARS optimizer를 사용하며 모든 batch size에 대해 안정적인 학습을 보여준다  

**Global BN**  
- 우리의 framework에서 positive pair는 같은 device에서 계산되므로 모델은 representation을 개선하지 않고도 local information leakage을 이용해 prediction accuracy를 개선시킬 수 있다   
- 우리는 학습하는 동안 모든 device에서 BN mean과 variance를 종합함으로써 이 issue를 해결했다  

### Evaluation Protocol  
**Dataset and Metrics**  
- SimCLR의 성능을 평가하기 위해 광범위한 dataset을 사용했다  
- linear classifier의 경우, base network의 weight를 frozen시킨 후 classifier layer만 학습을 했다  

**Default setting**  
- data augmentation으로 random crop and resize, random flip, color distortions 그리고 gaussian blur을 사용했다  
- base encoder로 ResNet-50을, projection head로 2-layer MLP를 사용해 128 dimension vector로 project했다  
- loss function으로 NT-Xent, optimizer로 LARS, learning rate는 4.8($=0.3\times BatchSize/256$)을 사용했다  
- batch size는 4096, epoch은 100을 설정했다  
- 처음 10 에폭에 대해서는 linear warmup을 사용하며 그 이후에는 cosine decay schedule을 사용했다

## Data Augmentation for Contrastive Representation Learning  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure3.png?raw=true" width=500>

**Data augmentation defines predictive tasks**  
- 현재 대부분의 approach들은 architecture를 바꿔서 contrastive prediction task를 정의한다  
- 예를 들자면, network architecture에서 receptive field를 제한해 global-to-local view prediction을 수행하는 것이다  
- 반면 fixed image splitting과 context aggregation network를 통해neighboring view prediction을 하는 것도 있다  
- 우리는 이러한 complexity를 target image의 random cropping을 수행함으로써 피할 수 있다는 것을 보일 것이다  

### Composition of data augmentation operations is crucial for learning good representations  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure4.png?raw=true" width=500>

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure5.png?raw=true" width=500>

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure6.png?raw=true" width=500>

- augmentation의 한 가지 type은 spatial, geometric transformation을 포함한다  
- 예를 들자면 cropping, resizing, rotation and cutout 등이 있다  
- 다른 type은 appearance transformation이며 예를 들면 color distortion, brightness, contrast, saturation 등이 있다  
- 각 augmentation의 효과와 중요성을 이해하기 위해 우리는 pair에 각 augmentation을 적용해서 실험한다  
- ImageNet의 경우 이미지 사이즈가 모두 다르기 때문에 항상 crop and resize를 적용해줘야 한다  
- 우리는 이 confound를 제거하기 위해 asymmetric data transformation setting을 고려한다  
- 한 이미지를 random crop을 적용하고 다른 이미지는 identity로 남겨둔다  
- figure 5는 transformation을 각각 적용한 결과를 보여준다  
- 우리는 single transformation은 good representation을 얻기에는 충분하지 않다는 것을 발견했다  
- 한 가지 눈에 띄는 점은, random cropping과 random color distortion이다  
- 우리가 추측하기로는, crop만 사용했을 때는 patch마다 color 분포가 비슷해서 심각한 문제가 생기게 된다  
- 그래서 일반화된 성능을 얻기 위해선 crop과 color distortion을 같이 구성하는 것이 중요하다  

### Contrastive learning needs stronger data augmentation than supervised learning  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table1.png?raw=true" width=500>

- stronger color augmentation은 성능을 상당히 개선해준다  
- augmentation의 set을 사용한 supervised model을 학습할 때 stronger color augmentation을 적용할 경우 성능을 떨어뜨리는 것을 발견했다  
- 따라서 color augmentation을 적용할 때 우리의 unsupervised contrastive learning이 supervised보다 더 성능이 좋다  

## Architectures for Encoder and Head  
### Unsupervised contrastive learning benefits (more) from bigger models   
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure7.png?raw=true" width=500>

- figure 7에서 모델의 사이즈가 클수록 unsupervised의 성능은 증가하는 것을 볼 수 있다  
- 우리는 supervised와 unsupervised로 학습한 linear classifier 사이의 gap은 model size를 증가함으로써 줄일 수 있는 것을 찾았다  

### A nonlinear projection head improves the representation quality of the layer before it  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/figure8.png?raw=true" width=500>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table3.png?raw=true" width=500>

- 우리는 projection head에 대해서 살펴봤다  
    - 1. identity mapping  
    - 2. linear projection  
    - 3. default nonlinear projection with one additional hidden layer (and ReLU activation)  
    
- nonlinear projection이 linear projection보다 약 3%, no projection 보다 약 10% 더 좋은 성능을 가진다  
- projection head에 대해서 output dimension에 상관 없이 비슷한 결과들을 보여준다  
- nonlinear projection을 사용할 때 hidden layer는 projection head의 전보다 후에 적용하는 것이 더 성능이 좋다  
- 이에 따른 이유를 추측하자면 head 이후에 hidden layer를 적용하면 contrastive loss의 손실이 일어나기 때문이라고 보고 있다  
- nonlinear transformation $g(\cdot)$을 조절함으로써 더 많은 information을 얻을 수 있으며 $\textbf[h}$를 유지할 수 있다  
- 이 가설을 확인하기 위해 우리는 pretraining하는 동안 적용된 transformation을 예측하는 것을 학습하기 위해 $\textbf{h}$ 또는 $g(\textbf{h})$를 사용하는 실험을 한다  

## Loss Functions and Batch Size  
### Normalized cross entropy loss with adjustable temperature works better than alternatives  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table2.png?raw=true" width=600>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table4.png?raw=true" width=500>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table5.png?raw=true" width=500>

- gradient를 보면서 우리는 2가지를 발견했다  
    - 1. temperature를 같이 쓴 $l_2$ normalization (cosine similarity)은 다른 example들에 효율적으로 weight를 주며 적절한 temperature는 hard negative를 학습하는 데 도움이 된다  
    - 2. cross entropy와 다른 objective function들은 relative hardness로 negative에 weight를 주지 않는다  
    
- 따라서 loss function은 semi-hard negative mining을 적용한다  
- 모든 loss term의 gradient를 계산하는 대신 semi-hard negative term을 사용해 loss를 계산한다  
- table 4에서 negative mining이 도움이 되지만 여전히 가장 좋은 결과는 NT-Xent loss보다 안좋은 성능을 보여준다  
- table 5에서 normalization과 적절한 temperature scaling을 하지 않으면 성능이 현저하게 떨어지는 것을 보이고 있다  
- $l_2$ normalization을 적용하지 않으면 contrastive task accuracy는 높아지지만 representation 결과는 linear evaluation에서 더 안좋다  

### Contrastive learning benefits (more) from larger batch sizes and longer training  
- figure 9에서 epochs, batch size에 따라 모델의 성능을 보여주고 있다  
- epoch을 100으로 설정할 때 batch size가 클수록 좋은 성능을 보여준다  
- epoch이 커질수록 각 batch size에 따른 모델의 성능 차이는 줄어든다  
- 일반적으로 batch size가 클수록 negative example이 많아지며 여기에 학습을 더 많이 할수록 더 좋은 representation을 얻게 된다  

## Comparison with State-of-the-art   
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table6.png?raw=true" width=500>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table7.png?raw=true" width=500>

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/SimCLR/figure/table8.png?raw=true" width=500>

- 우리는 ResNet-50 (3개의 hidden layer width multipliers of $1\times$, $2\times$, $4\times$)을 사용해서 SOTA와 비교를 한다  

**Linear evaluation**  
- 별도로 architecture를 디자인한 이전 method들에 비해 우리의 framework가 훨씬 더 나은 결과를 얻는다  
- ResNet-50 ($4\times$)가 가장 좋은 성능을 보여줬으며 supervised pretrained ResNet-50과 비슷한 성능을 보여준다  

**Semi-supervised learning**  
- ILSVRC-12 training dataset에서 labeling된 1% or 10% 정도를 sampling해서 사용했다  
- 우리의 framework가 SOTA method들보다 더 좋은 성능을 보여준다  

**Transfer learning**  
- supervised baseline보다 비슷하거나 우수한 성능을 보여준다  

## Conclusion  
- 우리는 contrastive visual representation learning을 위해 간단한 framework를 제안했다  
- 연구를 통해 우리가 찾은 finding들을 합쳐서 이전 method들보다 더 좋은 성능을 내도록 했다  
- 우리는 standard supervised learning에 비해 조금 다른 augmentation들을 사용했고 nonlinear head, 다른 loss function을 사용했다  
- 그 결과 간단한 아이디어만으로도 우리 framework는 좋은 성능을 보여준다